In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for measure in columns:
    print(measure['name'], measure['type'])

In [ ]:
engine.execute('select * from measurement limit 7').fetchall()

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from the last data point in the database
last_data_point = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(last_data_point)

In [ ]:
year_ago = dt.date(2017,8,23) - dt.timedelta(days= 365)
print(year_ago)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
date_prcp = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >=year_ago, Measurement.prcp != None).\
    order_by(Measurement.date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
date_prcp_df = pd.DataFrame(date_prcp, columns=['Date', 'Precipitation'])
date_prcp_df.set_index('Date', inplace=True)
date_prcp_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
date_prcp_df.plot(figsize=(15,10), linewidth=2, color='green')
plt.tight_layout()
plt.title('Annual Precipitation', fontsize=20)
plt.xlabel('Date', labelpad=15)
plt.ylabel('Inches', labelpad=15)
plt.savefig('Images/precipitation.png')

![precipitation](Images/precipitation.png)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
date_prcp_df.describe()

![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_count = session.query(Measurement.station).distinct().count()

station_count_df = pd.DataFrame({'Station Count':[station_count]})
station_count_df

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
most_active = session.query(Measurement.station,func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

most_active_df = pd.DataFrame(most_active, columns=['Station Name', 'Station Count'])
most_active_df

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
most_active_temps = session.query(func.min(Measurement.tobs),func.\
    max(Measurement.tobs),func.avg(Measurement.tobs)).filter(Measurement.\
    station == 'USC00519281').order_by(func.min(Measurement.tobs)).all()

most_active_temps_df = pd.DataFrame(most_active_temps, columns=['Low Temp', 'High Temp', 'Avg Temp'])
most_active_temps_df

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station
hist_temp = session.query(Measurement.tobs).filter(Measurement.\
    date >= year_ago, Measurement.station == 'USC00519281').\
    order_by(Measurement.tobs).all()

hist_temp_df = pd.DataFrame(hist_temp, columns=['Temperature Obersavations'])
hist_temp_df.head()

In [ ]:
# plot the results as a histogram
bins = 12
hist_temp_df.plot.hist(hist_temp,bins)
plt.ylim(0,70)
plt.savefig('Images/station-histogram.png')

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
prev_year_start = dt.date(2017,3,25) - dt.timedelta(days= 365)
prev_year_end = prev_year_start + dt.timedelta(days = 10)

calc_temp = calc_temps(prev_year_start,prev_year_end)
print(calc_temp)

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
ta_temp= list(np.ravel(calc_temp))
tmin = ta_temp[0]
tmax = ta_temp[2]
temp_avg = ta_temp[1]

plt.figure(figsize=(3,8))
plt.bar(1,temp_avg, color = 'coral',alpha = 0.5)
plt.errorbar(1,temp_avg, yerr=(tmax - tmin), color = 'k')
plt.ylim(0, 100)
plt.xticks([])
plt.title('Trip Avg Temp')
plt.ylabel('Temp (F)')
plt.tight_layout()
plt.savefig('Images/temperature.png')

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
rainfall_station = session.query(Station.station, Station.name, Station.latitude, 
        Station.longitude, Station.elevation, func.sum(Measurement.prcp)).\
        filter(Measurement.date >= prev_year_start, Measurement.date <= prev_year_end).\
        filter(Measurement.prcp != None).\
        filter(Station.station == Measurement.station).\
        group_by(Measurement.station).\
        order_by(func.sum(Measurement.prcp).desc()).all()

rainfall_df = pd.DataFrame(rainfall_station, columns=['Station', 'Station Name', 'Latitude', 'Longitude', 'Elevation', 'Total Rainfall'])
rainfall_df

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`
# Set the start and end date of the trip
# Use the start and end date to create a range of dates
# Stip off the year and save a list of %m-%d strings
# Loop through the list of %m-%d strings and calculate the normals for each date
start_date = dt.datetime.strptime('2017-03-25', "%Y-%m-%d")
end_date = dt.datetime.strptime('2017-04-04', "%Y-%m-%d")

normals = []
dt_list = []

iter_date = start_date
while iter_date <= end_date:
    dt_list.append(iter_date)
    st_str_dt = iter_date.strftime("%m-%d")
    iter_date += dt.timedelta(1)
    normals.append(*daily_normals(st_str_dt))

normals

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
normals_df = pd.DataFrame(normals, columns=['Trip Min','Trip Avg','Trip Max'])
normals_df['Date'] = pd.to_datetime(dt_list)
normals_df.set_index('Date', inplace=True)
normals_df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
normals_df.plot.area(stacked=False, alpha=0.2,figsize=(10,7))
plt.xlabel('Date')
plt.ylim(0, 80)
plt.tight_layout()
plt.savefig('Images/daily_normals.png')